**Prereqs**:
- Synapse Dedicated SQL Pool with control permissions
- ADLS Gen2 path with Storage Key or SAS
- Fabric Warehouse ready for COPY INTO
- Master key not yet created in source DB (run once).

In [ ]:
-- Parameters: edit for your environment
DECLARE @StorageAccountKey NVARCHAR(200) = '<storage-key-or-sas>';
DECLARE @ExternalBase NVARCHAR(400) = 'abfss://container@account.dfs.core.windows.net/fabric-export/'; -- trailing slash required
DECLARE @ExternalFileFormat NVARCHAR(128) = 'ff_parquet';
DECLARE @ExternalDataSource NVARCHAR(128) = 'eds_adls_export';
DECLARE @CetasCompression NVARCHAR(20) = 'snappy';
DECLARE @MaxTables INT = 0; -- 0 = all tables, else top N by size
DECLARE @MinRowCount BIGINT = 0; -- skip tiny tables if needed
DECLARE @SchemaFilter NVARCHAR(128) = NULL; -- set to filter to a single schema
SELECT 'Parameters loaded' AS status;

In [ ]:
-- Security objects: master key, credential, external source + file format
IF NOT EXISTS (SELECT 1 FROM sys.symmetric_keys WHERE name = '##MS_DatabaseMasterKey##')
BEGIN
    CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'ChangeThisStrongPassword!1';
END
IF NOT EXISTS (SELECT 1 FROM sys.database_credentials WHERE name = 'cred_adls_export')
BEGIN
    CREATE DATABASE SCOPED CREDENTIAL cred_adls_export
    WITH IDENTITY = 'Storage Account Key', SECRET = @StorageAccountKey;
END
IF NOT EXISTS (SELECT 1 FROM sys.external_data_sources WHERE name = @ExternalDataSource)
BEGIN
    CREATE EXTERNAL DATA SOURCE [
]
    WITH ( LOCATION = @ExternalBase, CREDENTIAL = cred_adls_export, TYPE = HADOOP );
END
IF NOT EXISTS (SELECT 1 FROM sys.external_file_formats WHERE name = @ExternalFileFormat)
BEGIN
    CREATE EXTERNAL FILE FORMAT [
]
    WITH ( FORMAT_TYPE = PARQUET, DATA_COMPRESSION = @CetasCompression );
END
PRINT 'Security + external objects ready.';

In [ ]:
-- Create helper schema + log tables (lightweight)
IF NOT EXISTS (SELECT 1 FROM sys.schemas WHERE name = 'migration') EXEC('CREATE SCHEMA migration');
IF OBJECT_ID('migration.cetas_log','U') IS NULL
BEGIN
    CREATE TABLE migration.cetas_log (
        log_id BIGINT IDENTITY(1,1) PRIMARY KEY,
        schema_name NVARCHAR(128),
        table_name NVARCHAR(256),
        status VARCHAR(20),
        rows_written BIGINT NULL,
        duration_seconds INT NULL,
        started_at DATETIME2 DEFAULT SYSUTCDATETIME(),
        finished_at DATETIME2 NULL,
        error_message NVARCHAR(MAX) NULL
    );
END;
PRINT 'Logging tables ready.';

In [ ]:
-- Generate CETAS statements into a temp table (ordered by size desc)
IF OBJECT_ID('tempdb..#cetas','U') IS NOT NULL DROP TABLE #cetas;
SELECT TOP (CASE WHEN @MaxTables = 0 THEN 2147483647 ELSE @MaxTables END)
        ROW_NUMBER() OVER (ORDER BY p.row_count DESC) AS seq,
        s.name AS schema_name,
        t.name AS table_name,
        CONCAT(
            'CREATE EXTERNAL TABLE [migration].[' , s.name, '_', t.name, '_cetas] WITH (
',
            '    LOCATION = ''', @ExternalBase, s.name, '/', t.name, '/'' ,
',
            '    DATA_SOURCE = ', @ExternalDataSource, ',
',
            '    FILE_FORMAT = ', @ExternalFileFormat, ',
',
            '    REJECT_TYPE = VALUE,
',
            '    REJECT_VALUE = 0
',
            ') AS SELECT * FROM [', s.name, '].[', t.name, '];'
        ) AS cetas_sql
    INTO #cetas
    FROM sys.tables t
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    JOIN sys.dm_db_partition_stats p ON p.object_id = t.object_id AND p.index_id IN (0,1)
    WHERE t.is_external = 0
      AND (@SchemaFilter IS NULL OR s.name = @SchemaFilter)
      AND p.row_count >= @MinRowCount
    GROUP BY s.name, t.name, p.row_count
    ORDER BY p.row_count DESC;
SELECT * FROM #cetas ORDER BY seq;

In [ ]:
-- Execute CETAS sequentially (adjust timeout via SET LOCK_TIMEOUT if needed)
DECLARE @seq INT = 1, @max INT = (SELECT COUNT(*) FROM #cetas);
DECLARE @sql NVARCHAR(MAX), @schema NVARCHAR(128), @table NVARCHAR(256);
WHILE @seq <= @max
BEGIN
    SELECT @sql = cetas_sql, @schema = schema_name, @table = table_name FROM #cetas WHERE seq = @seq;
    DECLARE @start DATETIME2 = SYSUTCDATETIME();
    BEGIN TRY
        EXEC(@sql);
        INSERT INTO migration.cetas_log(schema_name, table_name, status, rows_written, duration_seconds, finished_at)
        VALUES(@schema, @table, 'SUCCESS', NULL, DATEDIFF(SECOND, @start, SYSUTCDATETIME()), SYSUTCDATETIME());
    END TRY
    BEGIN CATCH
        INSERT INTO migration.cetas_log(schema_name, table_name, status, error_message, duration_seconds, finished_at)
        VALUES(@schema, @table, 'FAILED', ERROR_MESSAGE(), DATEDIFF(SECOND, @start, SYSUTCDATETIME()), SYSUTCDATETIME());
    END CATCH;
    SET @seq += 1;
END
SELECT * FROM migration.cetas_log ORDER BY log_id DESC;

In [ ]:
-- Emit COPY INTO statements for Fabric (run these in Fabric Warehouse)
SELECT
    CONCAT(
        'COPY INTO [', s.name, '].[', t.name, '] ',
        'FROM ''', @ExternalBase, s.name, '/', t.name, '/'' ',
        'WITH (
',
        '    FILE_TYPE = ''PARQUET'',
',
        '    CREDENTIAL=(IDENTITY=''Storage Account Key'', SECRET=''''', @StorageAccountKey, '''''),
',
        '    MAXERRORS = 0,
',
        '    TABLOCK = TRUE
',
        ');'
    ) AS copy_into_sql
FROM sys.tables t
JOIN sys.schemas s ON t.schema_id = s.schema_id
WHERE t.is_external = 0
  AND (@SchemaFilter IS NULL OR s.name = @SchemaFilter)
  AND t.name NOT LIKE 'migration_%'
ORDER BY s.name, t.name;

**Post-steps in Fabric Warehouse**:

1. Paste and run the generated COPY INTO statements in Fabric SQL endpoint.

2. Run statistics refresh (for example, `EXEC migration.sp_update_table_statistics` if deployed).

3. Validate row counts using `migration.data_load_validation` if you deploy the full optimization framework.
